In [1]:
import macropy.activate
import importlib
import sys
sys.path.append('../')
import torch
import torch.nn as nn
import torch.optim as optim
from Code.iff_macro import set_config
from Code.lstm import lstmPolicyPredictor, FullyConnected
from Code.envs.MountainCar import MultiMountainCar, LookupPolicy, PassiveEnv
from Code.SNN import RSNN, FeedForwardSNN, magicRSNN

In [2]:
config = {
    'ALPHA': 0,
    'BETA': 1,
    'RESET_ZERO': False,
    'THRESH_ADD': 1,
    'THRESH_DECAY': 1,
    'DECODING': 'potential',
    'SPIKE_FN': 'bellec',
    'SIM_TIME': 10
}
set_config(config)
import Code.macroNeurons as Neurons
importlib.reload(Neurons)

<module 'Code.macroNeurons' from '../Code/macroNeurons.py'>

In [3]:
env = PassiveEnv()

In [4]:
BATCH_SIZE = 64

In [5]:
#model = lstmPolicyPredictor(2,8,16)
#model = RSNN(config, 1, 32, 16, 1, Neurons.LIFNeuron, Neurons.AdaptiveNeuron, Neurons.OutputNeuron)
model = magicRSNN(config, 1, 32, 16, 1, Neurons.LIFNeuron, Neurons.MagicNeuron, Neurons.OutputNeuron)
#model = FeedForwardSNN(config, [1, 128, 128, 1], Neurons.LIFNeuron, Neurons.OutputNeuron)
#model = FullyConnected([1, 128, 128, 1])

teacher = LookupPolicy()


#TODO: test superspike instead of Bellec

In [6]:
spikes = torch.zeros((200, 10))
def logger(h, t, i):
    spikes[t, i] = h['spikes'][0].sum()
    

In [7]:
#bce = nn.BCELoss(reduction='none') #reduction='sum'
mse = nn.MSELoss(reduction='none')
optimizer = optim.Adam(model.parameters(), lr=0.0001)#0.00001

In [ ]:
for i in range(5000):
    model.zero_grad()
    inputs, targets, mask = env.getBatch(BATCH_SIZE)
    outputs, _ = model(inputs/0.4)
    #print(outputs.shape, targets.shape)
    loss = (mse(outputs.squeeze(dim=2)/20, targets) * mask).sum() / mask.sum()
    loss.backward()
    optimizer.step()
    if i%10 == 0:
        print(loss.item(), (loss/targets.view(-1).var()).item(), i) #, ((outputs>0.5) != targets).sum()
    if i%100 == 0:
        model(inputs/0.4, h=None, logger=logger)
        print(spikes)


0.06726884096860886 128.6168212890625 0
tensor([[ 0.,  0.,  0.,  ...,  6.,  8.,  7.],
        [11.,  8., 14.,  ..., 10., 12., 11.],
        [10., 12., 13.,  ..., 15., 15., 11.],
        ...,
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]], grad_fn=<CopySlices>)
0.06502585858106613 126.88727569580078 10
0.06226105988025665 115.61071014404297 20
0.060187071561813354 116.44641876220703 30
0.058090757578611374 104.29965209960938 40
0.05601292848587036 108.7671890258789 50
0.054401952773332596 103.85787963867188 60
0.05305013433098793 100.92911529541016 70
0.004703045357018709 9.360411643981934 80
0.004671634640544653 8.655961990356445 90


In [ ]:
#torch.save(model, '../models/lstm_passive')
#0.0002

In [ ]:
#torch.save(model, '../models/rsnn_passive_new1')
#0.001

In [ ]:
from matplotlib import pyplot as plt
#model = torch.load('../models/snn_passive3')
%matplotlib

In [ ]:
plt.figure()


In [ ]:
inputs[:, 0, 0].var().sqrt()

In [ ]:
model(inputs/0.4)[0].squeeze()

In [ ]:
def doplot():
    inputs, targets, mask = env.getBatch(1)
    outputs, _ = model(inputs/0.4)
    plt.close()
    plt.plot(inputs[:, 0, 0], targets)
    plt.plot(inputs[:, 0, 0], outputs.squeeze().detach()/20)

In [ ]:
doplot()